In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D,ZeroPadding2D
from keras.layers import Input, Lambda
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from keras.utils import plot_model
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Dense, Reshape, Activation, Multiply, Concatenate, Flatten, Lambda
from keras.models import Model, Sequential
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns



In [ ]:
# !nvidia-smi

In [ ]:
# physical_devices = tf.config.list_physical_devices('GPU') 
# # tf.config.set_visible_devices(physical_devices[0], 'GPU')
# tf.config.set_visible_devices([physical_devices[2]], 'GPU')

In [ ]:

# Define aspects of the model and create instances of both the 
# test and train batch generators and the complete model.

imsize = 28
batch_size = 32
embedding_dim = 2 
LR = 0.0001
EPOCHS = 5
alpha = 0.2 
input_x=20
input_y=125
total_classes=2

In [ ]:
import pickle

def get_image_count(file_path):
    # Open the file in binary mode for reading
    with open(file_path, 'rb') as f:
        # Load the data from the file using pickle
        loaded_data = pickle.load(f)

    # Check if the loaded data is a numpy array
    if isinstance(loaded_data, np.ndarray):
        # If loaded data is a numpy array, return the number of images (third dimension)
        return loaded_data.shape[2]
    else:
        # If loaded data is not a numpy array, return 0 (indicating no images)
        return 0

In [ ]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    else:
        print(f"Folder '{folder_path}' already exists.")

In [ ]:
import os
import random
from sklearn.model_selection import train_test_split
import time


global_path = ""
model_folder_name = "model_cbam" 

# model_path = f'{global_path}/model_history/{model_folder_name}'
model_path = f"model_history/{model_folder_name}"

create_folder_if_not_exists(model_path)
data_folder_path="data"
# history_path = os.path.join(global_path, f"model_history/{checkpoint_folder_name}/history/")
# checkpoint_path = os.path.join(global_path, f"model_history/{checkpoint_folder_name}/checkpoint/")
# Specify the directory path
X_seiz_train=[]
X_seiz_test=[]

X_bckg_train=[]
X_bckg_test=[]

image_count=20000  # How much images to load for trainig or testing

## Save model and logs

In [ ]:
emd_model_path = f"{model_path}/emd_{model_folder_name}.h5"
emd_history_log_path = f"{model_path}/emd_{model_folder_name}.csv"

cls_model_path = f"{model_path}/cls_{model_folder_name}.h5"
cls_history_log_path = f"{model_path}/cls_{model_folder_name}.csv"

In [ ]:
def reset_image_arr(test=0):
    global data_folder_path  # Assuming data_folder_path is defined globally

    # Initialize lists to store image data
    X_bckg_train, X_bckg_test, X_seiz_train, X_seiz_test = [], [], [], []

    # print("   --reset image started at", time.time(), test, len(X_bckg_train), len(X_bckg_test), len(X_seiz_train), len(X_seiz_test))

    if test == 2:
        # Define paths for seizure and background images in the test set
        seiz_folder_path_test = os.path.join(data_folder_path, "test/seiz")
        bckg_folder_path_test = os.path.join(data_folder_path, "test/bckg")
        seiz_file_names_test = os.listdir(seiz_folder_path_test)
        bckg_file_names_test = os.listdir(bckg_folder_path_test)

        # Load seizure images for test set
        while len(X_seiz_test) < image_count:
            i = random.randint(0, len(seiz_file_names_test) - 1)
            file_p = os.path.join(seiz_folder_path_test, seiz_file_names_test[i])
            with open(file_p, 'rb') as f:
                loaded_data = pickle.load(f)

            # Randomly select a fraction of images from the loaded data
            n = loaded_data.shape[2]     # Number of images in current pkl file
            x = min(1000, max(1, n // 4))
            idx_arr = set()
            while len(idx_arr) < x:
                idx_arr.add(random.randint(0, n - 1))
            for i in idx_arr:
                X_seiz_test.append(loaded_data[:, :, i])

        # Load background images for test set
        while len(X_bckg_test) < image_count:
            i = random.randint(0, len(bckg_file_names_test) - 1)
            file_p = os.path.join(bckg_folder_path_test, bckg_file_names_test[i])
            with open(file_p, 'rb') as f:
                loaded_data = pickle.load(f)

            # Randomly select a fraction of images from the loaded data
            n = loaded_data.shape[2]    # Number of images in current pkl file
            x = min(1000, max(1, n // 4))
            idx_arr = set()
            while len(idx_arr) < x:
                idx_arr.add(random.randint(0, n - 1))
            for i in idx_arr:
                X_bckg_test.append(loaded_data[:, :, i])

    else:
        # Define paths for seizure and background images in the train and validation sets
        seiz_folder_path_train = os.path.join(data_folder_path, "train/seiz")
        bckg_folder_path_train = os.path.join(data_folder_path, "train/bckg")
        seiz_folder_path_val = os.path.join(data_folder_path, "val/seiz")
        bckg_folder_path_val = os.path.join(data_folder_path, "val/bckg")

        seiz_file_names_train = os.listdir(seiz_folder_path_train)
        bckg_file_names_train = os.listdir(bckg_folder_path_train)
        seiz_file_names_val = os.listdir(seiz_folder_path_val)
        bckg_file_names_val = os.listdir(bckg_folder_path_val)

        # Load seizure images for training set
        while len(X_seiz_train) < image_count * 0.8:
            i = random.randint(0, len(seiz_file_names_train) - 1)
            file_p = os.path.join(seiz_folder_path_train, seiz_file_names_train[i])
            with open(file_p, 'rb') as f:
                loaded_data = pickle.load(f)

            # Randomly select a fraction of images from the loaded data
            n = loaded_data.shape[2]      # Number of images in current pkl file
            x = min(1000, max(1, n // 4))
            idx_arr = set()
            while len(idx_arr) < x:
                idx_arr.add(random.randint(0, n - 1))
            for i in idx_arr:
                X_seiz_train.append(loaded_data[:, :, i])

        # Load seizure images for validation set
        while len(X_seiz_test) < image_count * 0.2:
            i = random.randint(0, len(seiz_file_names_val) - 1)
            file_p = os.path.join(seiz_folder_path_val, seiz_file_names_val[i])
            with open(file_p, 'rb') as f:
                loaded_data = pickle.load(f)

            # Randomly select a fraction of images from the loaded data
            n = loaded_data.shape[2]       # Number of images in current pkl file
            x = min(1000, max(1, n // 4))
            idx_arr = set()
            while len(idx_arr) < x:
                idx_arr.add(random.randint(0, n - 1))
            for i in idx_arr:
                X_seiz_test.append(loaded_data[:, :, i])

        # Load background images for training set
        while len(X_bckg_train) < image_count * 0.8:
            i = random.randint(0, len(bckg_file_names_train) - 1)
            file_p = os.path.join(bckg_folder_path_train, bckg_file_names_train[i])
            with open(file_p, 'rb') as f:
                loaded_data = pickle.load(f)

            # Randomly select a fraction of images from the loaded data
            n = loaded_data.shape[2]       # Number of images in current pkl file
            x = min(1000, max(1, n // 4))
            idx_arr = set()
            while len(idx_arr) < x:
                idx_arr.add(random.randint(0, n - 1))
            for i in idx_arr:
                X_bckg_train.append(loaded_data[:, :, i])

        # Load background images for validation set
        while len(X_bckg_test) < image_count * 0.2:
            i = random.randint(0, len(bckg_file_names_val) - 1)
            file_p = os.path.join(bckg_folder_path_val, bckg_file_names_val[i])
            with open(file_p, 'rb') as f:
                loaded_data = pickle.load(f)

            # Randomly select a fraction of images from the loaded data
            n = loaded_data.shape[2]      # Number of images in current pkl file
            x = min(1000, max(1, n // 4))
            idx_arr = set()
            while len(idx_arr) < x:
                idx_arr.add(random.randint(0, n - 1))
            for i in idx_arr:
                X_bckg_test.append(loaded_data[:, :, i])

    # print("   --reset image successfully at", time.time(), test, len(X_bckg_train), len(X_bckg_test), len(X_seiz_train), len(X_seiz_test))
    return X_seiz_train, X_seiz_test, X_bckg_train, X_bckg_test

In [ ]:
def get_image(label, test=0):
    """
    Choose an image from our training or test data with the given label.

    Args:
        label (int): Label indicating the type of image (0 for background, 1 for seizure).
        test (int): Indicator for whether to select from test data (default 0 for training data).

    Returns:
        img: Selected image.
    """


    # Check label to determine whether to select background or seizure image
    if label == 0:  # Background image
        if test != 0:  # If test set
            idx = np.random.randint(len(X_bckg_test))  # Choose random index from test data
            img = X_bckg_test[idx]  # Select image
        else:  # If training set
            idx = np.random.randint(len(X_bckg_train))  # Choose random index from training data
            img = X_bckg_train[idx]  # Select image
    else:  # Seizure image
        if test != 0:  # If test set
            idx = np.random.randint(len(X_seiz_test))  # Choose random index from test data
            img = X_seiz_test[idx]  # Select image
        else:  # If training set
            idx = np.random.randint(len(X_seiz_train))  # Choose random index from training data
            img = X_seiz_train[idx]  # Select image
    
    # nan_positions = np.argwhere(np.isnan(img))
    # if len(nan_positions) > 0:
    #     print("heheheheheheheheheh")

    return img  


In [ ]:
def data_loader(test=0):
    """
    Generate an un-ending stream (i.e., a generator) of classification data for training or test.

    Args:
        test (int): Indicator for whether to generate data from test data (default 0 for training data).

    Yields:
        tuple: A tuple containing images and their corresponding labels.
    """
    global X_seiz_train, X_seiz_test, X_bckg_train, X_bckg_test

    # Infinite loop to continuously generate classification data
    limit = 0  # Counter for resetting image arrays
    while True:
        if limit % 200 == 0:
            # print("--Calling reset_image at ", time.time())  # Print timestamp when resetting images
            X_seiz_train, X_seiz_test, X_bckg_train, X_bckg_test = reset_image_arr(test)  # Reset image arrays

        a = []  # List to store images
        label = []  # List to store labels

        # Generate images and labels for each batch

        for _ in range(batch_size-4):
            img_class = np.random.randint(2)
            img = get_image(img_class, test)  # Get a image with label img_class
            a.append(img)
            label.append(img_class)  # Add label 0 for background image or 1 for seizure image
        
        img_0 = get_image(0, test)  # Get a image with label 0
        a.append(img_0)
        label.append(0)

        img_0 = get_image(0, test)  # Get a image with label 0
        a.append(img_0)
        label.append(0)

        img_1 = get_image(1, test)  # Get a image with label 1
        a.append(img_1)
        label.append(1)

        img_1 = get_image(1, test)  # Get a image with label 1
        a.append(img_1)
        label.append(1)


        # Convert lists to numpy arrays
        A = np.array(a, dtype='float32')
        label = np.array(label)

        # Shuffle the arrays while maintaining the correspondence between image and label
        rng_state = np.random.get_state()
        np.random.shuffle(A)
        np.random.set_state(rng_state)
        np.random.shuffle(label)
        label = tf.cast(label, dtype=tf.int32)
        # Yield the image data and labels
        yield [A], label

        # Increment limit counter
        limit += 1



In [ ]:
def mean_loss(labels, embeddings):
    ans = hard_triplet_loss(labels, embeddings, 0.2)
    return ans 

def triplet_lossss(x, alpha = 0.2):
    # Triplet Loss function.
    anchor,positive,negative = x
    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)
    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)
    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
    return loss

def hard_triplet_loss(label, emd, margin=0.2):
    # Compute pairwise distance matrix
    pairwise_dist = tf.norm(tf.expand_dims(emd, axis=1) - tf.expand_dims(emd, axis=0), axis=-1)

    # Get positive and negative mask
    label_equal = tf.equal(tf.expand_dims(label, axis=1), tf.expand_dims(label, axis=0))
    positive_mask = tf.cast(label_equal, dtype=tf.float32)
    negative_mask = 1.0 - positive_mask

    # Compute hardest positive and hardest negative distances
    max_positive_dist = tf.reduce_max(pairwise_dist * positive_mask, axis=1)
    min_negative_dist = tf.reduce_min((pairwise_dist + 1e6) * negative_mask, axis=1)

    # Compute triplet loss
    loss = tf.maximum(0.0, max_positive_dist - min_negative_dist + margin)

    # Compute mean loss over the batch
    mean_loss = tf.reduce_mean(loss)
    
    return mean_loss

def cbam_block(inputs, reduction_ratio=16):
    # Channel Attention
    channel = inputs.shape[-1]
    
    avg_pool = GlobalAveragePooling2D()(inputs)
    max_pool = GlobalMaxPooling2D()(inputs)
    
    avg_pool = Dense(channel // reduction_ratio, activation='relu')(avg_pool)
    avg_pool = Dense(channel)(avg_pool)
    
    max_pool = Dense(channel // reduction_ratio, activation='relu')(max_pool)
    max_pool = Dense(channel)(max_pool)
    
    channel_attention = Activation('sigmoid')(avg_pool + max_pool)
    channel_attention = Multiply()([inputs, Reshape((1, 1, channel))(channel_attention)])
    
    # Spatial Attention
    avg_pool = tf.reduce_mean(channel_attention, axis=-1, keepdims=True)
    max_pool = tf.reduce_max(channel_attention, axis=-1, keepdims=True)
    
    concat = Concatenate(axis=-1)([avg_pool, max_pool])
    spatial_attention = Conv2D(1, (7, 7), padding='same', activation='sigmoid')(concat)
    
    cbam_output = Multiply()([channel_attention, spatial_attention])
    return cbam_output

def embedding_model():
    input_layer = Input(shape=(input_x, input_y, 1), name='input_layer')
    
    x = Conv2D(32, (3, 3), activation='relu', name='convolution_1')(input_layer)
    x = Conv2D(64, (3, 3), activation='relu', name='convolution_2')(x)
    
    # Adding CBAM block after convolutional layers
    x = cbam_block(x)
    
    x = MaxPooling2D(pool_size=(2, 2), name='max_pooling')(x)
    x = Flatten(name='flatten')(x)
    x = Dense(256, activation='relu', name='dense_lol_1')(x)
    x = Dense(64, name='dense_lol_2')(x)
    output_layer = Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    return model



def complete_model(base_model):
    input_1 = Input((input_x, input_y,1), name='input_layer_A')
    A = base_model(input_1)

    model = Model(inputs=[input_1], outputs=A)
    model.compile(
        # loss=tfa.losses.TripletSemiHardLoss(),
                  loss = mean_loss,
                  optimizer=Adam(LR))
    return model
  

## Intialize data loader

In [ ]:

train_generator_emd = data_loader()
val_generator_emd = data_loader(test=1)

In [ ]:
next(train_generator_emd)

## Initialize Model Architecture 

In [ ]:
emd_model = embedding_model()
cmp_model = complete_model(emd_model)
cmp_model.summary()

In [ ]:
emd_model.summary()

## Training


In [ ]:

import csv
from keras.callbacks import Callback, ModelCheckpoint

# Custom callback to log loss and accuracy to CSV
class CSVLogger(Callback):
    def __init__(self, LOG_PATH):
        self.log_path = LOG_PATH


    def on_epoch_end(self, log_path, epoch, logs=None):
        logs = logs or {}
        with open(self.log_path, 'a', newline='') as f:
            writer = csv.writer(f)
            if epoch == 0:
                # Write the header
                writer.writerow(['epoch'] + list(logs.keys()))
            # Write the epoch number and metrics
            writer.writerow([epoch] + [logs[key] for key in logs.keys()])

In [ ]:
# Fit the model using triplet images provided by the train batch generator.
# Save the trained weights.


# Define a callback to save the model weights after each epoch
checkpoint_callback_emd = ModelCheckpoint(filepath=emd_model_path, 
                                      save_weights_only=False,
                                      verbose=1)



csv_logger_emd = CSVLogger(LOG_PATH=emd_history_log_path)

# Fit the model with the callbacks
history = cmp_model.fit_generator(train_generator_emd,
                                  validation_data=val_generator_emd,
                                  epochs=20,
                                  verbose=1,
                                  steps_per_epoch=1000,
                                  validation_steps=30,
                                  callbacks=[checkpoint_callback_emd, csv_logger_emd])




## Load Trained Model

In [ ]:
from keras.models import load_model

## Initialize Model instance

In [ ]:
emd_model_arc = embedding_model()
complete_model = complete_model(emd_model_arc)
complete_model.load_weights(emd_model_path)

In [ ]:
complete_model.summary()

## Load embedding model from complete model

In [ ]:
emd_model = embedding_model()

emd_model = complete_model.get_layer('model_2')

In [ ]:
emd_model.summary()

In [ ]:

def classification_model(embedding_model):
    """
    Create a classification model on top of the embedding model.
    
    Args:
    embedding_model: Pretrained embedding model
    
    Returns:
    classification_model: Model for classification on top of the embedding model
    """
    # Freeze the layers of the embedding model
    for layer in embedding_model.layers:
        layer.trainable = False
    
    model = Sequential()
    model.add(Input(shape=(input_x, input_y, 1), name='input_layer'))  # Input layer
    model.add(embedding_model)
    # model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='softmax'))  # 2 classes for binary classification
    
    return model



In [ ]:

# Create the classification model
classifier_model = classification_model(emd_model)

# Compile the model
classifier_model.compile(optimizer='adam',
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])

In [ ]:
classifier_model.summary()

## Initialize data loader

In [ ]:
X_seiz_train, X_seiz_test, X_bckg_train, X_bckg_test=reset_image_arr()
train_generator_cls=data_loader()
val_generator_cls=data_loader(test=1)

## Training classification model

In [ ]:
checkpoint_callback_cls = ModelCheckpoint(filepath=cls_model_path, 
                                      save_weights_only=False,
                                      verbose=1)



csv_logger_cls = CSVLogger(LOG_PATH=cls_history_log_path)

# Fit the model with the callbacks
history = classifier_model.fit_generator(train_generator_cls,
                                  validation_data=val_generator_cls,
                                  epochs=20,
                                  verbose=1,
                                  steps_per_epoch=5,
                                  validation_steps=2,
                                  callbacks=[checkpoint_callback_cls, csv_logger_cls])

## Load classification and embedding model

In [ ]:
emd_model_arc = embedding_model()
classifier_model_final = classification_model(emd_model_arc)
classifier_model_final.load_weights(cls_model_path)

In [ ]:
import pandas as pd
def test_complete_folder(pkl_folder_path, label,model):
    file_name_arr = os.listdir(pkl_folder_path)
    c_pkl = 0
    c_img = 0
    c_true_pkl = 0
    c_true_img = 0
    a=[]
    b=[]
    c=[]
    d=[]
    for i in range(len(file_name_arr)):
        file_name = file_name_arr[i]
        file_path = os.path.join(pkl_folder_path,file_name)
        with open(file_path, 'rb') as f:
            # Load the data from the file using pickle
            loaded_data = pickle.load(f)
        # print("-->",len(file_name_arr),i,loaded_data.shape[2])
        loaded_data=np.transpose(loaded_data, (2, 0, 1))
        loaded_data=np.expand_dims(loaded_data, axis=-1)
        # print("-->",len(file_name_arr),i,loaded_data.shape)
        y_pred = model.predict(loaded_data)

        y_pred = np.argmax(y_pred, axis=1)

        c_pkl+=1 
        c_img+=len(y_pred)

        curr_true = 0
        for curr_label in y_pred:
            if curr_label==label:
                c_true_img+=1 
                curr_true+=1 

        a.append(len(y_pred))
        b.append(curr_true)
        
        print(len(file_name_arr),i,"-------->>>",curr_true)
        if curr_true>len(y_pred)//2:
            c_true_pkl+=1 
            c.append(1)
        else:
            c.append(0)
        d.append(file_name)

        # break 
    df = pd.DataFrame({"file_name":d, "total_images":a, "true_predicted":b, "is_good_pkl":c})
    df.to_excel(f"model_history/{model_folder_name}/results_{label}.xlsx", index=False)
    return [c_img, c_pkl, c_true_img, c_true_pkl]



In [ ]:
num_img = 0
num_pkl = 0

correct_img = 0
correct_pkl = 0

bckg_path = "data/val/bckg"
seiz_path = "data/val/seiz"

In [ ]:
bckg_results  = test_complete_folder(bckg_path,0,classifier_model_final)

print(bckg_results)

In [ ]:
seiz_results  = test_complete_folder(seiz_path,1,classifier_model_final)

print(seiz_results)

In [ ]:
print("Accuracy img",(seiz_results[2]+bckg_results[2])/(seiz_results[0]+bckg_results[0]))
print("Accuracy pkl",(seiz_results[3]+bckg_results[3])/(seiz_results[1]+bckg_results[1]))

## Inferences for Image-level prediction

In [ ]:
true_labels_img = [1] * seiz_results[0] + [0] * bckg_results[0]  # assuming n and m are the counts of class 1 and class 2 images respectively

# Predicted labels
predicted_labels_img = [1] * seiz_results[2] + [0] * (seiz_results[0]-seiz_results[2]) +[0] * bckg_results[2] + [1] * (bckg_results[0]-bckg_results[2]) # assuming a and b are the counts of correct predictions for class 1 and class 2 respectively

# Create confusion matrix
conf_matrix = confusion_matrix(true_labels_img, predicted_labels_img)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix(image)")
plt.show()

# Compute and print classification report
print(classification_report(true_labels_img, predicted_labels_img))

## Patient level prediction

In [ ]:
true_labels_pkl = [1] * seiz_results[1] + [0] * bckg_results[1]  # assuming n and m are the counts of class 1 and class 2 images respectively

# Predicted labels
predicted_labels_pkl = [1] * seiz_results[3] + [0] * (seiz_results[1]-seiz_results[3]) +[0] * bckg_results[3] + [1] * (bckg_results[1]-bckg_results[3]) # assuming a and b are the counts of correct predictions for class 1 and class 2 respectively

# Create confusion matrix
conf_matrix = confusion_matrix(true_labels_pkl, predicted_labels_pkl)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix(pkl)")
plt.show()

# Compute and print classification report
print(classification_report(true_labels_pkl, predicted_labels_pkl))